<a href="https://colab.research.google.com/github/data-with-shobhit/Misc/blob/main/Web_Scrapping_Task_Amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*  ***REQUIREMENTS***  ***(NOTE: RUN ONLY ONE TIME BEFORE SCRAPPER)***

In [ ]:
import subprocess; print("Requirements Successful") if subprocess.run("pip install requests beautifulsoup4 pandas fake_useragent", shell=True).returncode == 0 else None


* ***WEB SCRAPPER***



In [ ]:
# Importing all neccesary libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent
from datetime import datetime
import time
import random



# Using headers to mimic real browser to avoid getting banned / restricted

ua = UserAgent()
headers = {
    'User-Agent': ua.random,
    'Accept-Language': 'en-US,en;q=0.9',
}


# Made a function to check response codes

status_codes = status_codes = {100: "Continue", 101: "Switching Protocols", 102: "Processing",
                               200: "OK", 201: "Created", 202: "Accepted", 204: "No Content",
                               301: "Moved Permanently", 302: "Found (Temporary Redirect)", 304: "Not Modified",
                               400: "Bad Request", 401: "Unauthorized", 403: "Forbidden", 404: "Not Found", 429: "Too Many Requests",
                               500: "Internal Server Error", 502: "Bad Gateway", 503: "Service Unavailable", 504: "Gateway Timeout"}

def fetch_url(url):

  response = requests.get(url, headers=headers)
  code=response.status_code
  if code in status_codes:
      print("*"*100)
      print(f"Response Code: {code} - {status_codes[code]}")
  else:
      print("*"*100)
      print(f"Response Code: {code} - Unknown Status")


# Define product category or name of product in this case we are using for laptop/laptops

search_product = "laptops"                                              # Change for different Brands eg . HP laptops , Dell laptops
base_url = f"https://www.amazon.in/s?k={search_product}"
fetch_url(base_url)
print()
print("*"*100)


# Function to extract product details
def get_laptop_data(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    laptops = []
    results = soup.find_all("div", {"data-component-type": "s-search-result"})

    for result in results:
        try:
            title = result.find("h2", class_="a-size-medium a-spacing-none a-color-base a-text-normal").text.strip()
        except AttributeError:
            title = None

        try:
            image = result.find("img", class_="s-image")["src"]
        except (AttributeError, TypeError):
            image = None

        try:
            rating = result.find("span", class_="a-icon-alt").text.strip()
        except AttributeError:
            rating = None

        try:
            price = result.find("span", class_="a-price-whole").text.strip()
        except AttributeError:
            price = None



        ad_badge = next((result.find("span", class_=cls)
        for cls in ["puis-label-popover-default", "puis-label-popover-hover","a-color-base", "a-link-normal aok-inline-block s-widget-sponsored-label-text"]
                         if result.find("span", class_=cls)), result.find("span", string="Sponsored"))

        ad_or_organic = "Ad" if ad_badge else "Organic"

        laptops.append({
            "Title": title,
            "Image": image,
            "Rating": rating,
            "Price": price,
            "Ad/Organic": ad_or_organic
        })

    return laptops


all_data = []
num_pages = 5 # Change this to scrape more pages minimum 5

for page in range(1, num_pages + 1):
    print(f"Scraping page ..... {page}")

    url = base_url + str(page)
    laptops = get_laptop_data(url)
    all_data.extend(laptops)
    time.sleep(2)  # add delay in seconds to avoid getting banned

print()
print("*"*100)

# Saving data to CSV with timestamp

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"amazon_laptops_{timestamp}.csv"
df = pd.DataFrame(all_data)
df.dropna(inplace=True)                  # Dropping all nan values
df.reset_index(drop=True, inplace=True)   # Resetting index to avoid Errors

df.to_csv(filename, index=False) # saving file to csv format

print(f"Data saved to {filename}")
print()
print("Task Completed Successfully!!")
print()
print("*"*100)


